In [1]:
import matplotlib.pyplot as plt
import numpy as np
import helpers as h
import pandas as pd

pygame 2.1.2 (SDL 2.0.16, Python 3.7.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def create_dataframe(filepath_data, filepath_setup):
    """
    Takes the data csv and the setup csv and combines them into 
    1 larger csv.
    """
    
    df_1 = pd.read_csv(filepath_data, sep=',')
    df_2 = pd.read_csv(filepath_setup, sep=',')
    
    
    #reference for concat: https://www.youtube.com/watch?v=iYWKfUOtGaw
    df_rl = pd.concat([df_1, df_2], axis=1)
    df_rl.columns = df_rl.columns.str.replace(' ', '')

    return df_rl

In [3]:
def filtered_df(df, cols, i, operator, pivot):
    if operator == ">":
        return (df[cols[i]] > pivot)
    else:
        return (df[cols[i]] <= pivot)

In [4]:
import random


def get_random_filters_given_columns(dataframe):
    """
    Using the column titles, and the min/max of the column, 
    this function returns a random boolean condition or filter
    """

    #     list of the columns
    #     lists of the min/max value of each column
    
    excluded_cols = ['wassuccesful',
                     'reward',
                     'actualsteps',
                     'targetsteps',
                     'runscompleted',
                     'length',
                     'height']
    
    cols = [col for col in dataframe.columns if col not in excluded_cols]
    i = random.randint(0,len(cols)-1)
    
    maxs = [dataframe[col].max() for col in cols]
    mins = [dataframe[col].min() for col in cols]

#     print(i)
#     print(maxs)
#     print(mins)
    
    filt_info = []
    pivot = random.randint(mins[i], 
                           maxs[i])
    b = random.randint(0,1)
    
    if b == 0:
#         filt = (dataframe[cols[i]] > pivot)
        filt = filtered_df(dataframe, cols, i, ">", pivot)
        
        filt_info.append((dataframe, cols, i, ">", pivot))
#         print(f"{cols[i]}>{pivot}")
    else:
#         filt = (dataframe[cols[i]] <= pivot)
        filt = filtered_df(dataframe, cols, i, "<=", pivot)
        filt_info.append((dataframe, cols, i, "<=", pivot))
#         print(f"{cols[i]}<={pivot}")
    
    
    return filt, filt_info




In [5]:
def average_reward_given_filters(dataframe, fltr):
    """
    Given a table of data (like a csv or a dataframe) it finds the average 
    reward after filtering the table
    """
    
    # filter the dataframe
    # reference fro loc: https://www.youtube.com/watch?v=Lw2rlcxScZY
    df_avg = dataframe.loc[fltr, 'reward']
#     print(pd.DataFrame(df_avg))
    
    
    # get the average of those that meet our conditions
    avg_reward = np.mean(df_avg)
    
    return avg_reward

In [6]:
# rl11
path = "/home/javonne/Uni/Orbital-Mechanics-game-with-AI/build/agents/2_star_2_planet/rl1"
filepath_data = f"/{path}/data_rlearn.csv"
filepath_setup = f"/{path}/setup_rlearn.csv"

In [7]:
df_rl = create_dataframe(filepath_data, filepath_setup)

In [8]:
filt, _ = get_random_filters_given_columns(df_rl)

In [9]:
average_reward_given_filters(df_rl, filt)

-168.99622641509433

In [10]:
def sub_domain_search(df_agent_1, agent_1, df_agent_2, agent_2):
    
    fitness_subdomain_winner = []
    
    # 1. You have two CSVs containing data that corresponds to two agents.
    # via params
    
    # you ensure each data tihng has the same columns
    if list(df_agent_1.columns) != list(df_agent_2.columns):
        raise Exception("These dataframes have different columns and cannot be compared. Perhaps the corresponding agents were trained with different inputs?")
    
        
    # 4. You create one or many random filters given the columns from the parameter, and the min/max of those columns.
    filt, filt_conditions = get_random_filters_given_columns(df_agent_1)

    #  check the performance of each agent in chosen filter
    agent_1_performance = average_reward_given_filters(df_agent_1, filt)
    agent_2_performance = average_reward_given_filters(df_agent_2, filt)

    difference = np.abs(agent_1_performance - agent_2_performance)

    # 6. You record all of the filters and which CSV has a higher reward (in a tuple like: Tuple[filter_conditions, better] ).
    if agent_1_performance >= agent_2_performance:
        fitness_subdomain_winner.append((difference, filt_conditions, agent_1))
    else:
        fitness_subdomain_winner.append((difference, filt_conditions, agent_2))

#     print(fitness_subdomain_winner)
#     print(fitness_subdomain_winner[0][0])
#     print(np.array(fitness_subdomain_winner).shape)
    
    # put biggest differences at the top of the list
#     fitness_subdomain_winner = fitness_subdomain_winner.sort()
#     fitness_subdomain_winner = fitness_subdomain_winner.reverse()
    
    return fitness_subdomain_winner



In [16]:
discovered_subdomain = sub_domain_search(df_rl, df_rl, 3)

TypeError: sub_domain_search() missing 1 required positional argument: 'agent_2'

In [13]:
discovered_subdomain[0][1][0]

NameError: name 'discovered_subdomain' is not defined

In [12]:
filtered_df(discovered_subdomain[0][1][0][0], discovered_subdomain[0][1][0][1], discovered_subdomain[0][1][0][2], discovered_subdomain[0][1][0][3], discovered_subdomain[0][1][0][4])

NameError: name 'discovered_subdomain' is not defined

In [ ]:
filt

In [ ]:
# psuedocode
# def sub_domain_search(csv_1, csv_2, [columns_to_search], number_of_checks):
#     filter_and_better = []
#     columns_1 = csv_1.columns
#     columns_2 = csv_2.columns
#     if columns_1 != columns_2:
#         raise Exception
#     for i in range(number_of_checks):
#         filter_1 = get_random_filters_given_columns(csv_1)
#         filter_2 = get_random_filters_given_columns(csv_2)
#         avg_1 = average_reward_given_filters(csv_1, filter_1)
#         avg_2 = average_reward_given_filters(csv_2, filter_2)
#         if avg_1 >= avg_2:
#             result = (filter_1, 0)
#         else:
#             result = (filter_2, 1)
#         filter_and_better.append(result)
#     return filter_and_better


In [ ]:
stuff = [(8,3,3),(6,5,5),(7,1,62)]
np.array(stuff).shape

In [ ]:
stuff

In [ ]:
stuff.sort()

In [ ]:
stuff.reverse()

In [ ]:
stuff[0][0]

In [ ]:
:

In [ ]:
def auto_sub_domain_search(df_agent_1, df_agent_2, num_checks):
    
    fitness_subdomain_winner = []
    
    # 1. You have two CSVs containing data that corresponds to two agents.
    # via params
    
    # you ensure each data tihng has the same columns
    if list(df_agent_1.columns) != list(df_agent_2.columns):
        raise Exception("These dataframes have different columns and cannot be compared. Perhaps the corresponding agents were trained with different inputs?")
    
    # 7. You do this check N times.
    for i in range(num_checks):
        
        # 4. You create one or many random filters given the columns from the parameter, and the min/max of those columns.
        filt, filt_conditions = get_random_filters_given_columns(df_agent_1)
        
        #  check the performance of each agent in chosen filter
        agent_1_performance = average_reward_given_filters(df_agent_1, filt)
        agent_2_performance = average_reward_given_filters(df_agent_2, filt)
        
        difference = np.abs(agent_1_performance - agent_2_performance)
        
        # 6. You record all of the filters and which CSV has a higher reward (in a tuple like: Tuple[filter_conditions, better] ).
        if agent_1_performance >= agent_2_performance:
            fitness_subdomain_winner.append((difference, filt_conditions, "agent_1"))
        else:
            fitness_subdomain_winner.append((difference, filt_conditions, "agent_2"))
    
#     print(fitness_subdomain_winner)
#     print(fitness_subdomain_winner[0][0])
#     print(np.array(fitness_subdomain_winner).shape)
    
    # put biggest differences at the top of the list
#     fitness_subdomain_winner = fitness_subdomain_winner.sort()
#     fitness_subdomain_winner = fitness_subdomain_winner.reverse()
    
    return fitness_subdomain_winner

